# Read and Explore Data

In [3]:
import sys

sys.path.append("../")

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from typing import Union, Tuple
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [4]:
from task1.retrieval_system import RetrievalSystem, SongInfo
from task1.similarity_measure import (
    cosine_similarity,
    dot_product,
    manhattan_distance,
    euclidean_distance,
    random_similarity,
)
from utils import read, embed_and_merge

In [5]:
# basic song information from task 1
df = read("information", 0)
df

,id,artist,song,album_name
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition)
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002)
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te
...,...,...,...,...
10090,zyzILCQvVeUFIINi,Crowded House,When You Come,Temple Of Low Men
10091,zzgS4ZqyswamEWNj,Britney Spears,My Only Wish (This Year),Platinum Christmas
10092,zzoFYDMlqU1X2zz1,Thundercat,DUI,Drunk
10093,zzpkRCGA5ud8q4mv,Otis Redding,Rock Me Baby,Otis Blue


In [6]:
# add genre information for metric calculation
genres = read("genres", 0)
# convert genre to actual list via eval
genres["genre"] = genres["genre"].apply(eval).apply(set)
df = df.merge(genres, on="id", how="left")

We load one new feature and the features from the previous assignments. We need them for comparison and fusion.

In [7]:
visual_feature = "resnet"
stats = read(visual_feature, 0)
df = embed_and_merge(df, stats, visual_feature)

for audio_feature in ["mfcc_bow", "blf_spectral", "ivec256", "musicnn"]:
    stats = read(audio_feature, 0)
    df = embed_and_merge(df, stats, audio_feature)

for text_feature in ["lyrics_bert", "lyrics_word2vec", "lyrics_tf-idf"]:
    stats = read(text_feature, 0)
    df = embed_and_merge(df, stats, text_feature.split("_")[1])

In [8]:
df.isna().sum()

id              0
artist          0
song            0
album_name      0
genre           1
resnet          1
mfcc_bow        1
blf_spectral    1
ivec256         1
musicnn         1
bert            0
word2vec        0
tf-idf          0
dtype: int64

In [9]:
# data for task 2 does not include the item with id "03Oc9WeMEmyLLQbj" = row 5
df = df.drop(5)
df = df.reset_index()

# Define retrieval systems

## From Task 1 (text-based)

In [10]:
rs_random = RetrievalSystem(
    df=df,
    sim_metric=random_similarity,
)

In [11]:
rs_cos_tdidf = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="tf-idf",
)

In [12]:
rs_cos_bert = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="bert",
)

In [13]:
rs_dot_w2v = RetrievalSystem(
    df=df,
    sim_metric=dot_product,
    sim_feature="word2vec",
)

## From Task 2 (audio-based)

In [14]:
rs_cos_mfcc = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="mfcc_bow",
)

In [15]:
rs_cos_blf = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="blf_spectral",
)

In [16]:
rs_cos_ivec256 = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="ivec256",
)

In [17]:
rs_cos_dnn = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="musicnn",
)

## From Task 3 (video-based; new!)


In [18]:
rs_cos_resnet = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="resnet",
)

## Fusion Techniques

### Early Fusion
In this section we wil perform early fusion by concatenating 2 features to form an aggregated feature


In [19]:
# simply concat features to form aggregated feature
text_feature = df["ivec256"]
audio_feature = df["mfcc_bow"]

combined_features = pd.concat([text_feature, audio_feature], axis=1)

combined_features['aggr_feature'] = combined_features.apply(lambda row: np.concatenate(row), axis=1)

print(f"Number of columns in the first feature: {len(combined_features.iloc[0, 0])}")
print(f"Number of columns in the second feature: {len(combined_features.iloc[0, 1])}")
print(f"Number of columns in the combined features: {len(combined_features.iloc[0, 2])}")


C:\Users\susig\AppData\Local\Temp\ipykernel_35224\66114822.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_features['aggr_feature'] = combined_features.apply(lambda row: np.concatenate(row), axis=1)


Number of columns in the first feature: 100
Number of columns in the second feature: 500
Number of columns in the combined features: 600


In [20]:
# scale features to mean=0, sd=1
scaler = StandardScaler()
arr= []

# conver to arr where feature values are columns and rows are samples
for row in combined_features["aggr_feature"]:
    arr.append(row)
arr =  np.array(arr)

# fit scaler to whole arr
scaler.fit(arr)

# transform aggregated feature
for row in combined_features["aggr_feature"]:
    scaler.transform(row.reshape(1, -1))

print(combined_features["aggr_feature"])

0        [-0.4192236661911011, -1.262790322303772, -0.3...
1        [1.4574661254882812, 0.619476318359375, -0.351...
2        [1.1996194124221802, -0.2549396753311157, 0.48...
3        [-0.6352253556251526, 0.6530497074127197, -1.6...
4        [-0.1336851567029953, 0.2777222692966461, -0.2...
                               ...                        
10089    [-0.7924780249595642, -0.1670402139425277, -0....
10090    [-0.5068019032478333, -0.5763552188873291, 0.5...
10091    [-0.6007039546966553, 0.0078461170196533, 1.47...
10092    [-0.8984915018081665, 0.3358610570430756, -0.6...
10093    [0.2389325052499771, -0.4458280205726624, -0.3...
Name: aggr_feature, Length: 10094, dtype: object


In [21]:
# use aggregated features for new RS
df["early_fusion"] = combined_features["aggr_feature"]

rs_cos_early_fusion = RetrievalSystem(
    df=df,
    sim_metric=cosine_similarity,
    sim_feature="early_fusion",
)

In [22]:
df

,index,id,artist,song,album_name,genre,resnet,mfcc_bow,blf_spectral,ivec256,musicnn,bert,word2vec,tf-idf,early_fusion
0,0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human,"{christian rock, rock}","[0.0, 0.110133, 0.31062, 0.0, 0.003017, 0.1360...","[1.7993406000000003, 0.0, 2.1731863, 0.0, 0.0,...","[-0.0249477, -0.02097, -0.0181111, -0.0154466,...","[-0.4192236661911011, -1.262790322303772, -0.3...","[0.12903129, 0.0011226882, 0.0065768533, 0.082...","[0.0302475523203611, 0.0352500043809413, 0.010...","[0.0193592727054678, 0.0232394714425702, 0.028...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0818293914712727, ...","[-0.4192236661911011, -1.262790322303772, -0.3..."
1,1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition),"{rap, grindcore, death metal, hip hop}","[0.016343, 0.029984, 0.018722, 0.0, 0.17283, 0...","[0.90309, 0.0, 2.5092025, 0.0, 1.2552725, 0.0,...","[-0.0309665, -0.0263328, -0.0233278, -0.021667...","[1.4574661254882812, 0.619476318359375, -0.351...","[0.026824217, 0.00087343465, 0.009360876, 0.31...","[0.0084422621876001, 0.0302564185112714, 0.009...","[0.018537292381979, 0.0113115924403394, 0.0107...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.4574661254882812, 0.619476318359375, -0.351..."
2,2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones,"{rock, pop punk}","[0.000348, 0.0, 1.073413, 0.0, 0.097732, 0.082...","[2.0086002, 0.0, 1.6334685, 0.0, 2.0413928, 0....","[-0.0329296, -0.0266801, -0.0224072, -0.020073...","[1.1996194124221802, -0.2549396753311157, 0.48...","[0.2518178, 0.0043474026, 0.07843659, 0.056584...","[0.0490818135440349, 0.0148476688191294, 0.001...","[0.0227837218553759, 0.0231641749730655, 0.012...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.1996194124221802, -0.2549396753311157, 0.48..."
3,3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002),"{progressive trance, trance, techno}","[1.747856, 1.07442, 0.618729, 0.03181, 0.09682...","[0.0, 0.0, 2.620136, 0.0, 0.0, 0.0, 2.4857213,...","[-0.0267741, -0.021581, -0.0176151, -0.0154089...","[-0.6352253556251526, 0.6530497074127197, -1.6...","[0.00045163534, 0.0013956887, 0.002990173, 0.8...","[0.0445394963026046, 0.0214906893670558, 0.013...","[0.0381116103401342, 0.0278804157207017, 0.016...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.6352253556251526, 0.6530497074127197, -1.6..."
4,4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te,"{water, ambient, pop rock, latin, pop folk, la...","[0.586726, 2.380147, 2.270726, 0.009559, 0.062...","[1.7160033, 0.0, 1.6232493, 0.0, 1.5440681, 0....","[-0.0312411, -0.0266276, -0.0242226, -0.022217...","[-0.1336851567029953, 0.2777222692966461, -0.2...","[0.21950364, 0.011686802, 0.054231934, 0.06058...","[0.0514551289379596, 0.0297695714980363, -0.01...","[0.0182936789026777, -0.0064870788035669, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.2413163920156013, ...","[-0.1336851567029953, 0.2777222692966461, -0.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089,10090,zyzILCQvVeUFIINi,Crowded House,When You Come,Temple Of Low Men,"{modern rock, pop rock, new wave, indie rock, ...","[0.494266, 0.477493, 0.284832, 0.000624, 0.028...","[1.1139433, 0.0, 1.2787536, 0.0, 1.1139433, 0....","[-0.0355863, -0.0310836, -0.0281663, -0.026088...","[-0.7924780249595642, -0.1670402139425277, -0....","[0.07816198, 0.028270144, 0.036110856, 0.16444...","[0.006713552866131, 0.0480893477797508, -0.001...","[0.0195101330379449, 0.0236336907562543, 0.011...","[0.0, 0.0, 0.079623055470056, 0.0, 0.0, 0.0, 0...","[-0.7924780249595642, -0.1670402139425277, -0...."
10090,10091,zzgS4ZqyswamEWNj,Britney Spears,My Only Wish (This Year),Platinum Christmas,"{soundtrack, teen pop, villancicos, rock, pop,...","[0.011208, 0.0, 0.261763, 0.0, 0.046742, 0.007...","[1.0, 0.0, 1.5314789, 0.0, 1.8692317, 0.0, 2.8...","[-0.0312189, -0.0272694, -0.02426

### Late Fusion
In this section we will perform late fusion of 2 retrieval systems using score aggregation by:
- precomputing all retrievals and their similarities for chosen retrieval systems.
- checking statistical compatability of scores
- fusing systems via score average

In [23]:
# pre compute retrievals
#res = []
#
#
#for system in [rs_cos_dnn, rs_cos_resnet]:
#    sim_matrix = pd.DataFrame(
#        index=df["id"], columns=df["id"], dtype=float)
#    id_order = sim_matrix.index
#    for idx, song_id in enumerate(df["id"]):
#        sim = system.retrieve(song_id, len(df["id"])).set_index("id")["similarity"]
#        # remove sort
#        sim = sim.loc[id_order]
#        # Fill the original DataFrame with the new DataFrame column
#        sim_matrix.iloc[:, idx] = sim
#    print(sim_matrix)
#    res.append(sim_matrix)

In [24]:
## check compatability
#for i, sim_ma in enumerate(res):
#    print(f"\nStatistics for res[{i}]:")
#    print(f"  Mean: {sim_ma.mean().mean()}")
#    print(f"  Standard Deviation: {sim_ma.std().mean()}")

In [25]:
# create pd dataframe consisting of all retrieval systems, with metric and feature
rs = pd.DataFrame(
    [
        ["random", "random", rs_random],
        ["cosine", "tf-idf", rs_cos_tdidf],
        ["cosine", "bert", rs_cos_bert],
        ["dot", "word2vec", rs_dot_w2v],
        ["cosine", "mfcc_bow", rs_cos_mfcc],
        ["cosine", "blf_spectral", rs_cos_blf],
        ["cosine", "ivec256", rs_cos_ivec256],
        ["cosine", "musicnn", rs_cos_dnn],
        ["cosine", "resnet", rs_cos_resnet],
        ["cosine", "early_fusion", rs_cos_early_fusion],
        
    ],
    columns=["metric", "feature", "rs_object"],
)

In [26]:
rs

,metric,feature,rs_object
0,random,random,<task1.retrieval_system.RetrievalSystem object...
1,cosine,tf-idf,<task1.retrieval_system.RetrievalSystem object...
2,cosine,bert,<task1.retrieval_system.RetrievalSystem object...
3,dot,word2vec,<task1.retrieval_system.RetrievalSystem object...
4,cosine,mfcc_bow,<task1.retrieval_system.RetrievalSystem object...
5,cosine,blf_spectral,<task1.retrieval_system.RetrievalSystem object...
6,cosine,ivec256,<task1.retrieval_system.RetrievalSystem object...
7,cosine,musicnn,<task1.retrieval_system.RetrievalSystem object...
8,cosine,resnet,<task1.retrieval_system.RetrievalSystem object...
9,cosine,early_fusion,<task1.retrieval_system.RetrievalSystem object...


# Evaluation

In [27]:
from pipeline import Pipeline

evaluate = Pipeline(rs, genres)

Creating genre overlap matrix:   0%|          | 0/10094 [00:00<?, ?it/s]

Creating result lists for every rs (max_k=100):   0%|          | 0/10 [00:00<?, ?it/s]

calculating results for system.metric='random' system.feature='random':   0%|          | 0/10094 [00:00<?, ?it…

calculating results for system.metric='cosine' system.feature='tf-idf':   0%|          | 0/10094 [00:00<?, ?it…

calculating results for system.metric='cosine' system.feature='bert':   0%|          | 0/10094 [00:00<?, ?it/s…

...

calculating results for system.metric='cosine' system.feature='blf_spectral':   0%|          | 0/10094 [00:00<…

calculating results for system.metric='cosine' system.feature='ivec256':   0%|          | 0/10094 [00:00<?, ?i…

calculating results for system.metric='cosine' system.feature='musicnn':   0%|          | 0/10094 [00:00<?, ?i…

calculating results for system.metric='cosine' system.feature='resnet':   0%|          | 0/10094 [00:00<?, ?it…

calculating results for system.metric='cosine' system.feature='early_fusion':   0%|          | 0/10094 [00:00<…

In [28]:
evaluate.load_results_csv("task_3.csv")
result = evaluate.run(
    [
        # (Pipeline.mean_precision_at_k, dict(k=10)),
        # (Pipeline.mean_recall_at_k, dict(k=10)),
        # (Pipeline.precision_and_recall_interval, dict(k_min=1, k_max=100, step_size=5)),
        # (Pipeline.mean_ndcg_at_k, dict(k=10)),
        # (Pipeline.genre_coverage_at_k, dict(k=10)),
        (Pipeline.mean_genre_diversity_at_k, dict(k=10)),
    ]
)
result.to_csv("task_3.csv", index=False)
result

FileNotFoundError: [Errno 2] No such file or directory: 'task_3.csv'

In [ ]:
for rs in result.itertuples():
    metrics_df = rs.precision_and_recall_interval

    plt.figure(figsize=(12, 8))
    plt.plot(
        metrics_df["recall"],
        metrics_df["precision"],
        marker="o",
        label=f"{rs.metric=} ({rs.feature=})",
    )

    # Annotate each point with its k value
    for k, recall, precision in zip(
        metrics_df.index, metrics_df["recall"], metrics_df["precision"]
    ):
        plt.annotate(
            f"k={k}",
            (recall, precision),
            textcoords="offset points",
            xytext=(0, 10),
            ha="center",
        )

    plt.title(f"Precision-Recall Curve for {rs.metric=} with {rs.feature=}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.grid(True)
    plt.show()